In [1]:
# 匯入必要的套件
import xgboost as xgb
from sklearn.datasets import load_iris ,load_breast_cancer
from sklearn.model_selection import train_test_split ,StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

# 分割資料集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# 使用XGBClassifier並進行訓練
model = xgb.XGBClassifier(objective='multi:softmax', num_class=3, max_depth=3, eta=0.1, eval_metric='merror')
model.fit(X_train, y_train)

# 預測
y_pred = model.predict(X_test)

# 計算準確率
accuracy = accuracy_score(y_test, y_pred)
print(f"模型的準確率: {accuracy * 100:.2f}%")

# 輸出分類報告
report = classification_report(y_test, y_pred, target_names=iris.target_names)
print("\n分類報告:")
print(report)

# 輸出混淆矩陣
conf_matrix = confusion_matrix(y_test, y_pred)
print("\n混淆矩陣:")
print(conf_matrix)

模型的準確率: 97.33%

分類報告:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        29
  versicolor       0.92      1.00      0.96        23
   virginica       1.00      0.91      0.95        23

    accuracy                           0.97        75
   macro avg       0.97      0.97      0.97        75
weighted avg       0.98      0.97      0.97        75


混淆矩陣:
[[29  0  0]
 [ 0 23  0]
 [ 0  2 21]]


# StratifiedKFold

StratifiedKFold 是 sklearn 中的一個交叉驗證方法，專門用於處理分類問題。它與 KFold 相似，但它在進行資料分割時特別注重維持每個類別在每一折中的比例，這對於不平衡的資料集非常有用。

**StratifiedKFold 的特點**：

1.   類別比例不變：StratifiedKFold 確保在每一折中，每個類別的樣本比例與原始資料集中保持一致。例如，假設你有 3 個類別，每個類別的樣本數量佔比不同，StratifiedKFold 會盡量確保在每一折的訓練集與測試集中，這些類別的分佈比例是相同的。
2.   適用於分類問題：這個方法特別適合分類問題，特別是當類別分佈不平衡時，可以避免某些折中無法代表所有類別的問題。

**舉例：**


*   假設你有一個二元分類的資料集，其中 90% 的樣本屬於類別 0，而 10% 的樣本屬於類別 1。如果使用普通的 KFold，可能會出現某一折中只有類別 0 的樣本，而完全沒有類別 1，這會影響模型的訓練與評估。
*   使用 StratifiedKFold，可以確保在每一折中，類別 0 和類別 1 的樣本比例與原始資料集一致，例如，如果原始資料集中有 90% 的樣本屬於類別 0，那麼在每一折中也會保持大約 90% 的樣本屬於類別 0，10% 屬於類別 1。




In [2]:
# 載入Iris資料集
iris = load_iris()
X = iris.data
y = iris.target

# 建立5折交叉驗證
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 初始化XGBClassifier
model = xgb.XGBClassifier(objective='multi:softmax', num_class=3, max_depth=3, eta=0.1, eval_metric='merror')

# 進行交叉驗證
fold = 1
for train_index, test_index in skf.split(X, y):
    # 切分資料
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # 訓練模型
    model.fit(X_train, y_train)

    # 預測
    y_pred = model.predict(X_test)

    # 計算準確率
    accuracy = accuracy_score(y_test, y_pred)

    # 輸出分類報告和混淆矩陣
    report = classification_report(y_test, y_pred, target_names=iris.target_names)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # 印出結果
    print(f"Fold {fold}:")
    print(f"準確率: {accuracy * 100:.2f}%")
    print("\n分類報告:")
    print(report)
    print("\n混淆矩陣:")
    print(conf_matrix)
    print("-" * 50)

    fold += 1

Fold 1:
準確率: 96.67%

分類報告:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       0.91      1.00      0.95        10
   virginica       1.00      0.90      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30


混淆矩陣:
[[10  0  0]
 [ 0 10  0]
 [ 0  1  9]]
--------------------------------------------------
Fold 2:
準確率: 96.67%

分類報告:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       0.91      1.00      0.95        10
   virginica       1.00      0.90      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30


混淆矩陣:
[[10  0  0]
 [ 0 10  0]
 [ 0  1  9]]
---------------------------------------------

In [3]:
# 載入較大的資料集 - Breast Cancer 資料集
data = load_breast_cancer()
X = data.data
y = data.target

# 建立5折交叉驗證
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 初始化XGBClassifier
model = xgb.XGBClassifier(objective='binary:logistic', max_depth=3, eta=0.1, eval_metric='error')

# 進行交叉驗證
fold = 1
for train_index, test_index in skf.split(X, y):
    # 切分資料
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # 訓練模型
    model.fit(X_train, y_train)

    # 預測
    y_pred = model.predict(X_test)

    # 計算準確率
    accuracy = accuracy_score(y_test, y_pred)

    # 輸出分類報告和混淆矩陣
    report = classification_report(y_test, y_pred, target_names=data.target_names)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # 印出結果
    print(f"Fold {fold}:")
    print(f"準確率: {accuracy * 100:.2f}%")
    print("\n分類報告:")
    print(report)
    print("\n混淆矩陣:")
    print(conf_matrix)
    print("-" * 50)

    fold += 1

Fold 1:
準確率: 96.49%

分類報告:
              precision    recall  f1-score   support

   malignant       0.95      0.95      0.95        43
      benign       0.97      0.97      0.97        71

    accuracy                           0.96       114
   macro avg       0.96      0.96      0.96       114
weighted avg       0.96      0.96      0.96       114


混淆矩陣:
[[41  2]
 [ 2 69]]
--------------------------------------------------
Fold 2:
準確率: 92.98%

分類報告:
              precision    recall  f1-score   support

   malignant       0.95      0.86      0.90        43
      benign       0.92      0.97      0.95        71

    accuracy                           0.93       114
   macro avg       0.93      0.92      0.92       114
weighted avg       0.93      0.93      0.93       114


混淆矩陣:
[[37  6]
 [ 2 69]]
--------------------------------------------------
Fold 3:
準確率: 96.49%

分類報告:
              precision    recall  f1-score   support

   malignant       1.00      0.90      0.95        42
  